In [34]:
import tweepy
import sys
import pandas as pd
import datetime
consumer_key = "k0bMQjP8ChDxhVxT5XyYUJdIE"
consumer_secret = "HczZWqoHIDm0Y7aBMAKsN8pHT93qowu2y5g9n5f9R8GxrtoIt4"
access_token = "1182276910811881477-DvX7DdChb1OrfeK9ampR9qQdeufVSu"
access_token_secret = "74DZ0bKv2xnQXKOED977gSudqOHeq9NUew1n4CpYo7cDL"

In [35]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) 

In [36]:
# The Twitter user who we want to get tweets from
name = "Nigel_Farage"

In [37]:
# Number of tweets to pull
tweetCount = 20

# Calling the user_timeline function with our parameters
results = api.user_timeline(id=name, count=tweetCount, tweet_mode = 'extended')

# printing the text stored inside the tweet object
tweets = [[tweet.full_text] for tweet in results]

print(tweets[0])
timestamp_list = [tweet.created_at for tweet in results]
print(timestamp_list[:3])

['Boris Johnson says that the new EU treaty gets Brexit done, but Mr Barnier says the next negotiations will last for 3 years or longer. Who do you believe?']
[datetime.datetime(2019, 10, 23, 9, 14, 55), datetime.datetime(2019, 10, 22, 18, 56, 24), datetime.datetime(2019, 10, 22, 18, 0, 54)]


In [38]:
df = pd.DataFrame(columns = ['user_name', 'tweet_text', 'reply_list', 
                             'user_followers', 'user_location', 'user_verified',
                             'like_count', 'rt_count', 'timestamp'])

In [39]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

In [ ]:
tweets = []
i = 0
file_name = 'brexit-data'
non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)  

for tweet in tweepy.Cursor(api.search, q='brexit', lang="en", tweet_mode='extended', timeout=999999).items(): 
    if (datetime.datetime.now() - tweet.created_at).days < 30:
        print(i, end='\r')
        df.loc[i, 'tweet_text'] = tweet.full_text
        df.loc[i, 'user_name'] = tweet.user.name
        df.loc[i, 'reply_list'] = None
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'user_location'] = tweet.user.location
        df.loc[i, 'user_verified'] = tweet.user.verified
        df.loc[i, 'like_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'timestamp'] = tweet.created_at
        df.to_csv('{}.csv'.format(file_name))
        i+=1


Rate limit reached. Sleeping for: 263


In [25]:
df.head()

,user_name,tweet_text,reply_list,user_followers,user_location,user_verified,like_count,rt_count,timestamp
0,margaret hennessy,RT @KTHopkins: I will have a BREXIT UPDATE for...,None,169,,False,0,123,2019-10-23 11:54:35
1,Hen Wraig Ddig #FBPE,RT @BrechinYes2: Ian Blackford asks Johnston w...,None,583,"United Kingdom, Wales",False,0,36,2019-10-23 11:54:35
2,Dougie Graham,RT @Nigel_Farage: Boris Johnson says that the ...,None,425,,False,0,904,2019-10-23 11:54:35
3,Gertie Pye,RT @Shining09807978: Right here !!! \n\nAnna S...,None,186,,False,0,282,2019-10-23 11:54:35
4,James Lawler,"RT @Doozy_45: Hillary Benn:\n""When we get to t...",None,3057,United Kingdom,False,0,217,2019-10-23 11:54:35


## 

In [8]:
print(tweets[0])

user: 'Nigel Farage', tweet: 'Boris Johnson says that the new EU treaty gets Brexit done, but Mr Barnier says the next negotiations will last for 3 years or longer. Who do you believe?'
 reply_list: '@Nigel_Farage This Brexit never will happen by deal  - no deal and good bye EU
@Nigel_Farage After 31/10 there is no further negotiation on Brexit
@Nigel_Farage @suzydymna Its an Incredible dishonor to the People of the UK that Voted for Brexit.
Another 3 Years For what?
More multi year delays!

The people who Voted for Brexit Deserve Better.
@Nigel_Farage If there's another referendum and ime presented with Boris's deal or remaining, what should I do? If ime presented with a GE and I have to choose Brexit Party or Tory because I definately don't want Corbyn, what do I vote for.
@Nigel_Farage Who do you believe Nigel? Seems to me you've told us to never believe a word Mr Barnier has ever said, however now that it suits you we should.  We need you to just stand opposite the labour marginals

In [13]:
print(tweets[0].reply_list[1])

@Nigel_Farage They'll be a General Election before that mate so make sure you get ready. We need the list of ppc in… https://t.co/bbu1C6oonJ
